In [21]:
import numpy as np
import tensorflow as tf
from sklearn.datasets import load_iris
from sklearn.model_selection import train_test_split
from sklearn.preprocessing import OneHotEncoder
from sklearn.metrics import roc_auc_score, accuracy_score
from keras.models import Sequential
from keras.layers import Dense
from keras.optimizers import Adam

iris_data = load_iris() # load the iris dataset

print('Example data: ')
print(iris_data.data[:5])
print('Example labels: ')
print(iris_data.target[:5])

x = iris_data.data
y_ = iris_data.target.reshape(-1, 1) # Convert data to a single column

# One Hot encode the class labels
encoder = OneHotEncoder(sparse=False)
y = encoder.fit_transform(y_)
#print(y)

# Split the data for training and testing
train_x, test_x, train_y, test_y = train_test_split(x, y, test_size=0.20)

Example data: 
[[5.1 3.5 1.4 0.2]
 [4.9 3.  1.4 0.2]
 [4.7 3.2 1.3 0.2]
 [4.6 3.1 1.5 0.2]
 [5.  3.6 1.4 0.2]]
Example labels: 
[0 0 0 0 0]


/usr/local/lib/python3.6/dist-packages/sklearn/preprocessing/_encoders.py:415: FutureWarning: The handling of integer data will change in version 0.22. Currently, the categories are determined based on the range [0, max(values)], while in the future they will be determined based on the unique values.
If you want the future behaviour and silence this warning, you can specify "categories='auto'".
In case you used a LabelEncoder before this OneHotEncoder to convert the categories to integers, then you can now use the OneHotEncoder directly.
  warnings.warn(msg, FutureWarning)


In [23]:
num_classes = train_y.shape[1]
num_features = train_x.shape[1]
num_output = train_y.shape[1]
num_layers_0 = 10
num_layers_1 = 10
starter_learning_rate = 0.01
print(num_classes, num_features, num_output, train_x.shape)

3 4 3 (120, 4)


In [0]:
input_X = tf.placeholder('float32',shape =(None,num_features),name="input_X")
input_y = tf.placeholder('float32',shape = (None,num_classes),name='input_Y')

In [0]:
weights_0 = tf.Variable(tf.random_normal([num_features,num_layers_0], stddev=(1/tf.sqrt(float(num_features)))))
bias_0 = tf.Variable(tf.random_normal([num_layers_0]))
weights_1 = tf.Variable(tf.random_normal([num_layers_0,num_layers_1], stddev=(1/tf.sqrt(float(num_layers_0)))))
bias_1 = tf.Variable(tf.random_normal([num_layers_1]))
weights_2 = tf.Variable(tf.random_normal([num_layers_1,num_output], stddev=(1/tf.sqrt(float(num_layers_1)))))
bias_2 = tf.Variable(tf.random_normal([num_output]))

In [0]:
def forward_pass(input_x, input_y):

  hidden_output_0 = tf.nn.relu(tf.matmul(input_x,weights_0)+bias_0)
  hidden_output_1 = tf.nn.relu(tf.matmul(hidden_output_0,weights_1)+bias_1)
  predicted_y = tf.matmul(hidden_output_1,weights_2) + bias_2
  loss = tf.reduce_mean(tf.nn.softmax_cross_entropy_with_logits_v2(logits=predicted_y,labels=input_y))
  
  return predicted_y, loss

predicted_y, loss = forward_pass(input_X, input_y)

In [0]:
## Variable learning rate
learning_rate = tf.train.exponential_decay(starter_learning_rate, 0, 5, 0.85, staircase=True)
## Adam optimzer for finding the right weight
# optimizer = tf.train.GradientDescentOptimizer(learning_rate).minimize(loss,var_list=[weights_0,weights_1,weights_2, bias_0,bias_1,bias_2])
optimizer = tf.train.AdamOptimizer(learning_rate).minimize(loss,var_list=[weights_0,weights_1,weights_2,
                                                                         bias_0,bias_1,bias_2])

In [0]:
## Metrics definition
correct_prediction = tf.equal(tf.argmax(train_y,1), tf.argmax(predicted_y,1))
accuracy = tf.reduce_mean(tf.cast(correct_prediction, tf.float32))

In [31]:
## Training parameters
s = tf.InteractiveSession()

batch_size = 5
epochs=100
training_accuracy = []
training_loss = []
testing_accuracy = []
s.run(tf.global_variables_initializer())
for epoch in range(epochs):    
    arr = np.arange(train_x.shape[0])
    np.random.shuffle(arr)
    for index in range(0,train_x.shape[0],batch_size):
        s.run(optimizer, {input_X: train_x[arr[index:index+batch_size]],
                          input_y: train_y[arr[index:index+batch_size]],
                         })
    training_accuracy.append(s.run(accuracy, feed_dict= {input_X:train_x, 
                                                         input_y:train_y}))
    training_loss.append(s.run(loss, {input_X: train_x, 
                                      input_y: train_y}))
    
    ## Evaluation of model
    testing_accuracy.append(accuracy_score(test_y.argmax(1), 
                            s.run(predicted_y, {input_X: test_x}).argmax(1)))
    print("Epoch:{0}, Train loss: {1:.2f} Train acc: {2:.3f}, Test acc:{3:.3f}".format(epoch,
                                                                    training_loss[epoch],
                                                                    training_accuracy[epoch],
                                                                   testing_accuracy[epoch]))

/usr/local/lib/python3.6/dist-packages/tensorflow/python/client/session.py:1735: UserWarning: An interactive session is already active. This can cause out-of-memory errors in some cases. You must explicitly call `InteractiveSession.close()` to release resources held by the other session(s).
  warnings.warn('An interactive session is already active. This can '


Epoch:0, Train loss: 1.00 Train acc: 0.333, Test acc:0.367
Epoch:1, Train loss: 0.73 Train acc: 0.683, Test acc:0.633
Epoch:2, Train loss: 0.50 Train acc: 0.825, Test acc:0.733
Epoch:3, Train loss: 0.42 Train acc: 0.683, Test acc:0.733
Epoch:4, Train loss: 0.32 Train acc: 0.850, Test acc:0.833
Epoch:5, Train loss: 0.23 Train acc: 0.975, Test acc:0.933
Epoch:6, Train loss: 0.19 Train acc: 0.958, Test acc:0.933
Epoch:7, Train loss: 0.15 Train acc: 0.975, Test acc:1.000
Epoch:8, Train loss: 0.12 Train acc: 0.975, Test acc:1.000
Epoch:9, Train loss: 0.16 Train acc: 0.933, Test acc:0.933
Epoch:10, Train loss: 0.20 Train acc: 0.908, Test acc:0.900
Epoch:11, Train loss: 0.39 Train acc: 0.808, Test acc:0.767
Epoch:12, Train loss: 0.09 Train acc: 0.975, Test acc:0.967
Epoch:13, Train loss: 0.16 Train acc: 0.933, Test acc:0.900
Epoch:14, Train loss: 0.08 Train acc: 0.975, Test acc:0.967
Epoch:15, Train loss: 0.11 Train acc: 0.950, Test acc:0.900
Epoch:16, Train loss: 0.22 Train acc: 0.892, Test 